# DB Creation

Creating db from the data extracted pdf

In [1]:
from pdf2image import convert_from_path
import os

from dotenv import load_dotenv
# loading variables from .env file
load_dotenv() # pass a path if it's not a .env in the current working directory 
 
# accessing and printing value
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
maxima_pdf = "pdfs/MAXIMA_KC13_2025_QPRZ.pdf"
rimi_pdf = "pdfs/Rimi_Piedvjumi_2503-31032025.pdf"
top_pdf = "pdfs/veikals top.pdf"

In [3]:
maxima = convert_from_path(maxima_pdf)

In [4]:
rimi = convert_from_path(rimi_pdf)

In [5]:
top = convert_from_path(top_pdf)

In [6]:
maxima_prompt = """Write me in text what you extract. Your task is to extract from the brochure the name of the products and DISCOUNT prices for them. Do as follows - Trauku mazgāšanas līdzeklis FAIRY LEMON 1 iepak./2x 1350ml (2.22 euro/l) 5.99 euro. I am interested ONLY in regular discounts, do not take discounts with a customer card!

change the order, write like this - Trauku mazgāšanas līdzeklis FAIRY LEMON 1 iepak./2x 1350ml 5.99€ (2.22 €/l)

make it in a table. the table should be divided into columns such as name, discount price, price per kg/l/gab
if value for column of price per kg/l/gab then return N/A

Only provide table, do not say anything else
"""

In [7]:
rimi_prompt = """Could you please identify each product's name, its advertised price, as well as the price per kg/l/gab, and put it in a table? Omit any products from the table if they do not have a price listed. 
make it in a table. the table should be divided into columns such as name, discount price, price per kg/l/gab
if value for column of price per kg/l/gab then return N/A
Only provide table, do not say anything else

"""

In [8]:
top_prompt = """ Could you please identify each product's name, its advertised price, as well as the price per unit, and put it in a table? Omit any products from the table if they do not have a price listed. 

Could you please take into consideration decimals and place them in the correct location, and also consistently make use of the euro sign in the table.

If there is no price per unit listed, then please use the volume listed and advertised price and calculate the price per unit.

To identify the relationship between the listed price and the product is refers to, you can find the name of the product in the image of the product. The price sticker that overlaps with the product image will be the correct advertised price.

Please double check that the price per unit is listed for those items whose volume is listed as 1kg, and make sure that the advertised price is then listed under price per unit as “[advertised price] per kg”.
make it in a table. the table should be divided into columns such as name, discount price, price per kg/l/gab
if value for column of price per kg/l/gab then return N/A
Only provide table, do not say anything else

"""

In [9]:
from PIL import Image
from google import genai

client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[maxima[0], maxima_prompt])
print(response.text)

| Name                                                               | Discount price | Price per kg/l/gab |
| :------------------------------------------------------------------- | :--------------- | :----------------- |
| Veļas mazgāšanas kapsulas PERSIL 1 iepak./68 gab. + K2R 3 m.r.      | 14.99€           | 0.22€/gab          |
| Trauku mazgāšanas līdzeklis FAIRY LEMON 1 iepak./2x 1350ml         | 5.99€            | 2.22€/l            |
| Garie gurķi plēvē 1 gab.                                             | 0.79€            | N/A               |
| Siers MAASDAM 1 kg, 45%                                              | 6.99€            | N/A                |
| Tomāti mini MIX spainītī 500 g                                      | 1.99€            | 3.98€/kg           |
| Mazie apelsīni NAVELINA 1 kg                                       | 0.99€            | N/A                |
| Skābais krējums SMILTENE 400 g, 20%                                | 1.29€            | 3.23€/kg         

In [10]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[top[0], top_prompt])
print(response.text)

| Name                                    | Discount Price | Price per kg/l/gab |
|-----------------------------------------|----------------|--------------------|
| Cūkgaļa KURZEMES GAĻSAIMNIEKS SENČU    | €6.99           | €6.99 per kg        |
| Baltmaize DRUVA ZELTA MĀKOŅMAIZE      | €1.09           | €2.42 per kg        |
| Piens ALMA 2.5%                         | €0.95           | €0.95 per l         |
| Kviešu milti DOBELE EKSTRA              | €1.65           | €0.83 per kg        |
| Konservēta barība kaķiem SHEBA           | €1.69           | €4.97 per kg        |
| Alus PIEBALGAS                          | €1.09           | €2.18 per l         |
| Čipsi ĀDAŽU KRAUKŠĶI SIERA BUMBAS      | €1.49           | €9.03 per kg        |
| Sausā barība suņiem DARLING | N/A | N/A |


In [11]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[rimi[0], rimi_prompt])
print(response.text)

| Name | Discount Price | Price per kg/l/gab |
|---|---|---|
| Cūkgaļas un liellopu gaļas maltā masa, Forevers, 1 kg | 3.99€ | N/A |
| Atlantijas laša fileja, no 3-4 kg laša, 1 kg | 14.99€ | N/A |
| Olas RIMI, Nr. 1, A/LM, 10 gab. | 1.99€ | 0.21 €/gab |
| Saldkrējuma sviests CESVAINE, 82%, 180 g | 1.99€ | 11.06 €/kg |
| Šokolādes batoniņi SNICKERS, 10 x 50 g | 3.89€ | 7.78 €/kg |
| Konservi kaķiem ar mājputnu gaļu SHEBA, 12 x 85 g | 4.99€ | 4.89 €/kg |
| Gaišais alus UŽAVAS, 4,6%, 0,5 l | 1.49€ | 2.98 €/l |


In [20]:
import sqlite3
import re

# Connect to SQLite database
conn = sqlite3.connect("grocery.db")
cursor = conn.cursor()

# Create tables following the provided schema
cursor.executescript("""
CREATE TABLE IF NOT EXISTS Stores (
    StoreID INTEGER PRIMARY KEY AUTOINCREMENT,
    StoreName TEXT UNIQUE
);

CREATE TABLE IF NOT EXISTS GroceryItems (
    ItemID INTEGER PRIMARY KEY AUTOINCREMENT,
    ItemName TEXT UNIQUE
);

CREATE TABLE IF NOT EXISTS Prices (
    PriceID INTEGER PRIMARY KEY AUTOINCREMENT,
    ItemID INTEGER,
    StoreID INTEGER,
    Price DECIMAL(10,2),
    PricePerQuantity DECIMAL(10,2) NULL,
    FOREIGN KEY (ItemID) REFERENCES GroceryItems(ItemID),
    FOREIGN KEY (StoreID) REFERENCES Stores(StoreID)
);
""")

conn.commit()


In [21]:
import sqlite3
import re

# Connect to SQLite database
conn = sqlite3.connect("grocery.db")
cursor = conn.cursor()

# Function to clean and convert price strings to decimals
def parse_price(price_str):
    if price_str and "N/A" not in price_str:
        cleaned_price = re.sub(r"[^\d.]", "", price_str)
        if cleaned_price.endswith('.'):
            cleaned_price = cleaned_price[:-1]  # Remove trailing dot
        try:
            return round(float(cleaned_price), 2)
        except ValueError:
            return None #in case there are still some errors, we return None.
    return None

# Function to parse response text into a structured list
def parse_response_text(response_text):
    lines = response_text.split("\n")[2:]  # Skip table headers
    parsed_data = []

    for line in lines:
        parts = [p.strip() for p in line.split("|")[1:-1]]  # Remove empty edges
        if len(parts) == 3:
            name, price, price_per_quantity = parts
            parsed_data.append((name.lower(), price, price_per_quantity))
    
    return parsed_data

# Function to insert data dynamically into the database
def insert_data(store_name, response_text):
    # Parse the response text into structured data
    products = parse_response_text(response_text)

    # Insert store if not exists
    cursor.execute("INSERT OR IGNORE INTO Stores (StoreName) VALUES (?)", (store_name,))
    conn.commit()
    
    # Get StoreID
    cursor.execute("SELECT StoreID FROM Stores WHERE StoreName = ?", (store_name,))
    store_id = cursor.fetchone()[0]

    for product in products:
        item_name, price, price_per_quantity = product

        # Convert prices to decimals
        price = parse_price(price)
        price_per_quantity = parse_price(price_per_quantity)

        # Insert product if not exists
        cursor.execute("INSERT OR IGNORE INTO GroceryItems (ItemName) VALUES (?)", (item_name,))
        conn.commit()

        # Get ItemID
        cursor.execute("SELECT ItemID FROM GroceryItems WHERE ItemName = ?", (item_name,))
        item_id = cursor.fetchone()[0]

        # Insert price record
        cursor.execute("""
            INSERT INTO Prices (ItemID, StoreID, Price, PricePerQuantity)
            VALUES (?, ?, ?, ?)
        """, (item_id, store_id, price, price_per_quantity))

    conn.commit()

In [22]:
def calling_model_and_storing(data, store_name, prompt):
    for i in range(1):
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[data[i], prompt])
        insert_data(store_name, response.text)

In [23]:
calling_model_and_storing(maxima, 'maxima', maxima_prompt)

In [24]:
calling_model_and_storing(rimi, 'rimi', rimi_prompt)


In [25]:
calling_model_and_storing(top, 'top', top_prompt)